# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [21]:
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV

In [22]:
df = pd.read_csv('/datasets/users_behavior.csv')
df

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [24]:
for i in df.columns: 
    print(df[i].unique())
df[['minutes', 'mb_used']].describe()

[ 40.  85.  77. 106.  66.  58.  57.  15.   7.  90.  82.  45.  51.  56.
 108.   6.   2.  26.  79.  49.  93.  48.  11.  53.  81. 154.  37.  50.
  41.  10.  71.  65. 110. 120.  76.  64.  23.  34.  98.  35.   5.  70.
 124. 129.  67.   0.  13.  68.  91. 121. 114. 125.  80.  33. 138.  84.
  78.  69.  63.  72.  73.   1.  43. 118.  74.  83. 141. 117.  54. 101.
  29.   3. 107.  55.  47. 158.  87.  28.  59.  52.  44.  17. 111. 109.
  14.  92.  94.  46. 133.  75.  38.  60. 100.  31.  61.  89.  27. 196.
  24.  99.  62. 162. 116. 123.  18.  21.  12.  86.  32.  95.  39.  30.
  25.  36.  42. 113.   9. 183. 156. 127.  96.  16.   4. 102.  97.  20.
 104. 144.  19. 132. 131. 136.  88. 115. 176. 160. 164. 169.  22. 105.
 152. 177. 161. 112.   8. 126. 178. 103. 130. 198. 119. 137. 150. 122.
 146. 151. 157. 203. 143. 148. 128. 185. 167. 181. 184. 171. 153. 140.
 159. 188. 134. 189. 182. 173. 172. 145. 180. 155. 174. 244. 165. 163.
 142. 168.]
[311.9  516.75 467.66 ... 634.44 462.32 566.09]
[ 83.  56.  86.  

,minutes,mb_used
count,3214.000000,3214.000000
mean,438.208787,17207.673836
std,234.569872,7570.968246
min,0.000000,0.000000
25%,274.575000,12491.902500
50%,430.600000,16943.235000
75%,571.927500,21424.700000
max,1632.060000,49745.730000


Ошибок в данных не обнаружено.

Столбцы calls и messages необходимо перевести в int. 

In [25]:
for i in ['calls', 'messages']:
    df[i] = df[i].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int64  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int64  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 125.7 KB


## Разбейте данные на выборки

Разобьем данные на тренировочную, валидационную, тестовые выборки в соотношении - 0.6, 0.2, 0.2 соответственно.

In [26]:
features = df.drop(
    'is_ultra',
    axis = 1   
)

target = df['is_ultra']

In [7]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
   features, target, test_size=0.2, random_state=1)

In [27]:
features_train, features_valid, target_train, target_valid = train_test_split(
   features_train_valid, target_train_valid, test_size=0.25, random_state=1)

In [9]:
print('features_train', len(features_train))
print('features_valid', len(features_valid))
print('features_test', len(features_valid))

features_train 1928
features_valid 643
features_test 643


Имеется нужное соотношение

## Исследуйте модели

RandomForestClassifier LogisticRegression DecisionTreeClassifier

DecisionTreeClassifier

In [28]:
best_model_DTC = 0 
best_accuracy_DTC = 0
for i in range (1, 15): 
    model_DTC = DecisionTreeClassifier(
                                        max_depth=i, 
                                        random_state=1,
                                    )
    
    model_DTC.fit(features_train, target_train)
    prediction = model_DTC.predict(features_valid)
    accuracy = accuracy_score(target_valid, prediction)
    if accuracy > best_accuracy_DTC: 
        best_accuracy_DTC = accuracy
        best_model_DTC = i
print(best_model_DTC, best_accuracy_DTC)
    

4 0.7978227060653188


LogisticRegression

In [29]:
model_LR = LogisticRegression()
model_LR.fit(features_train, target_train)
prediction = model_LR.predict(features_valid)
accuracy_LR = accuracy_score(target_valid, prediction)
print(accuracy_LR)

0.7558320373250389


RandomForestClassifier

In [12]:
best_model_RFC = 0 
best_accuracy_RFC = 0
for depth in range (1, 15): 
    for n_tree in (10, 51, 10):
        model_RFC = RandomForestClassifier(
            max_depth=depth, 
            n_estimators=n_tree,
            random_state=1
        )
        
        model_RFC.fit(features_train, target_train)
        prediction = model_RFC.predict(features_valid)
        accuracy = accuracy_score(target_valid, prediction)
        if accuracy > best_accuracy_RFC: 
            best_accuracy_RFC = accuracy
            best_model_RFC = f'n_tree: {n_tree}, depth: {depth}'
print(best_model_RFC, best_accuracy_RFC)

n_tree: 51, depth: 9 0.8102643856920684


## Проверьте модель на тестовой выборке

Обучим модели с найденными гиперпараметрами на объединенной выборке из тестовой и валидационной и проверим их эффективность на тестовой выборке.

In [13]:
model_DTC = DecisionTreeClassifier(
                                    max_depth=4, 
                                    random_state=1
                                  )
model_DTC.fit(features_train_valid, target_train_valid)
prediction = model_DTC.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

0.776049766718507


In [30]:
model_LR = LogisticRegression()
model_LR.fit(features_train_valid, target_train_valid)
prediction = model_LR.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

0.6842923794712286


In [31]:
model_RFC = RandomForestClassifier(
        max_depth=9, 
        n_estimators=51,
        random_state=1
    )
model_RFC.fit(features_train_valid, target_train_valid)
prediction = model_RFC.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

0.7916018662519441


accuracu больший чем 0.75 демонстрирует дерево с глубиной 4, случайный лес с глубиной 9 и количеством деревьев 51. 

## (бонус) Проверьте модели на адекватность

In [32]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

In [33]:
print(985/df['is_ultra'].value_counts().sum())

0.30647168637212197


Так как соотношение тарифов смарт и ультра 70 к 30, то указанные выше модели можно считать адекватными, так как их accuracu 0.77 и 0.79. То есть выбор тарифа smart во всех случаях даст более низкую эффективность. 

Попробуем решить эти задачи c помощью GridSearchCV

In [34]:
tree = DecisionTreeClassifier()
parameters = {'max_depth': list(range(1, 25))}
grid_tree = GridSearchCV(tree, parameters, cv=5, scoring='accuracy')
grid_tree.fit(features_train_valid, target_train_valid)
print(grid_tree.best_params_)
print(grid_tree.best_score_)

prediction = grid_tree.best_estimator_.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

{'max_depth': 10}
0.7961882815156208
0.7807153965785381


In [35]:
forest = RandomForestClassifier()
parameters = {'max_depth': list(range(1, 25)), 'n_estimators':list(range(1, 51, 10))}
grid_forest = GridSearchCV(forest, parameters, cv=5, scoring='accuracy')
grid_forest.fit(features_train_valid, target_train_valid)
print(grid_forest.best_params_)
print(grid_forest.best_score_)

prediction = grid_forest.best_estimator_.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

{'max_depth': 17, 'n_estimators': 41}
0.8097986475765933
0.807153965785381


In [36]:
logist = LogisticRegression()
parameters = {'penalty': ['none', 'l2', 'l1']}
grid_logist = GridSearchCV(logist, parameters, cv=5, scoring='accuracy')
grid_logist.fit(features_train_valid, target_train_valid)
print(grid_logist.best_params_)
print(grid_logist.best_score_)

prediction = grid_logist.best_estimator_.predict(features_test)
accuracy = accuracy_score(target_test, prediction)
print(accuracy)

{'penalty': 'none'}
0.7425250273884628
0.6842923794712286


/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 1306, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py", line 443, in _check_solver
    raise ValueError("Solver %s supports only 'l2' or 'none' penalties, "
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  warnings.warn("Estimator fit failed. The score on this train-test"
/opt/conda/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:610: Fi

GridSearchCV дал результаты отличающейся и более эффективные чем ранее использованный метод перебора при помощи циклов. Возможно, из-за использования кросс-валидации.

Вывод. 
Предсказание перехода пользователей на тарифы smart и ultra с accuracy более 0.75 возможно. 

<div class="alert alert-info"> Спасибо за дополнительную информацию!

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [ ] Весь код исполняется без ошибок
- [ ] Ячейки с кодом расположены в порядке исполнения
- [ ] Выполнено задание 1: данные загружены и изучены
- [ ] Выполнено задание 2: данные разбиты на три выборки
- [ ] Выполнено задание 3: проведено исследование моделей
    - [ ] Рассмотрено больше одной модели
    - [ ] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [ ] Написаны выводы по результатам исследования
- [ ] Выполнено задание 3: Проведено тестирование
- [ ] Удалось достичь accuracy не меньше 0.75
